## Fraud Detection with Azure OpenAI & Saving Results to Lakehouse (Gold Layer)
This guide shows how to:

  - Use Azure OpenAI to generate fraud summaries and risk scores for transactions

  - Mask extremely sensitive columns

  - Save the final results to the Gold layer in the Lakehouse (as a single CSV file)

#### Step 1: Load your transaction data into a Pandas DataFrame


In [ ]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Tables/Silver/bank_transactions_data/part-00000-5ee6fa47-1fec-453c-af80-50954400a764-c000.snappy.parquet"
df = pd.read_parquet("/lakehouse/default/Tables/Silver/bank_transactions_data/part-00000-5ee6fa47-1fec-453c-af80-50954400a764-c000.snappy.parquet")
display(df)

In [ ]:
!pip install openai

In [ ]:
df = df.drop(columns=["TransactionDate", "PreviousTransactionDate"], errors="ignore")

In [ ]:
df

,TransactionID,AccountID,TransactionAmount,TransactionType,Location,DeviceID,IP_Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance
0,TX000001,AC00128,14.09,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21
1,TX000002,AC00455,376.24,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91
2,TX000003,AC00019,126.29,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35
3,TX000004,AC00070,184.50,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06
4,TX000005,AC00411,13.45,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,TX002508,AC00297,856.21,Credit,Colorado Springs,D000625,21.157.41.17,M072,Branch,33,Doctor,109,1,12690.79
2508,TX002509,AC00322,251.54,Debit,Tucson,D000410,49.174.157.140,M029,Branch,48,Doctor,177,1,254.75
2509,TX002510,AC00095,28.63,Debit,San Diego,D000095,58.1.27.124,M087,Branch,56,Retired,146,1,3382.91
2510,TX002511,AC00118,185.97,Debit,Denver,D000634,21.190.11.223,M041,Online,23,Student,19,1,1776.91


### Step 2: Enrich Data with Azure OpenAI (Using JSON-Based Prompts) to get fraud summary + risk score
Each transaction is sent individually to Azure OpenAI with a structured prompt that expects a JSON response. This avoids parsing issues and ensures reliability.

In [ ]:
import pandas as pd
import openai
import json
import time
from tqdm import tqdm

# Set Azure OpenAI credentials
azure_api_key = "<your-api-key>"
azure_endpoint = "https://<your-resource-name>.openai.azure.com/"


client = openai.AzureOpenAI(
    api_key=azure_api_key,
    api_version="2024-02-01",
    azure_endpoint=azure_endpoint,
    azure_deployment="gpt-4o-kenya-hack"
)

# Model Deployment
gpt_deployment = "gpt-4o-kenya-hack"  

# 3. Convert timestamps to strings (avoids JSON errors) 
for col in df.columns:
    if pd.api.types.is_datetime64_any_dtype(df[col]):
        df[col] = df[col].astype(str)

# 4. GPT Call per Transaction with Retry 
def analyze_transaction(transaction, max_retries=3):
    for attempt in range(max_retries):
        try:
            prompt = f"""
You are a fraud detection expert. Given a transaction record, return a JSON with:
- "FraudSummary": a 1-2 sentence fraud analysis.
- "FraudRisk": "Low", "Medium", or "High" risk level.

Transaction: {json.dumps(transaction)}

Return JSON like: {{"FraudSummary": "...", "FraudRisk": "High"}}
            """
            response = client.chat.completions.create(
                model="gpt-4o-kenya-hack",
                messages=[
                    {"role": "system", "content": "You only respond in JSON."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=300
            )
            content = response.choices[0].message.content
            parsed = json.loads(content)

            # Ensure required keys are present
            if "FraudSummary" in parsed and "FraudRisk" in parsed:
                return parsed
        except Exception:
            time.sleep(1)

    # Final fallback if all attempts fail — skip this row
    raise ValueError("GPT failed to return valid JSON after retries.")

#  5. Analyze All Rows 
results = []
skipped_indices = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    transaction_dict = row.to_dict()
    try:
        result = analyze_transaction(transaction_dict)
        results.append(result)
    except Exception as e:
        print(f"[Warning] Skipped row {idx} due to error: {e}")
        skipped_indices.append(idx)
        results.append({"FraudSummary": "", "FraudRisk": ""})  
    time.sleep(1.2)  

# 6. Merge Results ---
results_df = pd.DataFrame(results)
df["FraudSummary"] = results_df["FraudSummary"]
df["FraudRisk"] = results_df["FraudRisk"]

  0%|          | 0/2512 [00:00<?, ?it/s]

[Warning] Skipped row 0 due to error: GPT failed to return valid JSON after retries.


  0%|          | 1/2512 [00:10<7:01:43, 10.08s/it]

[Warning] Skipped row 1 due to error: GPT failed to return valid JSON after retries.


  0%|          | 2/2512 [00:17<5:54:59,  8.49s/it]

...

100%|█████████▉| 2506/2512 [4:15:50<00:34,  5.75s/it]

100%|█████████▉| 2507/2512 [4:15:58<00:32,  6.50s/it]

100%|█████████▉| 2508/2512 [4:16:02<00:22,  5.60s/it]

100%|█████████▉| 2510/2512 [4:16:08<00:08,  4.47s/it]

100%|█████████▉| 2511/2512 [4:16:23<00:07,  7.73s/it]

In [ ]:
df

,TransactionID,AccountID,TransactionAmount,TransactionType,Location,DeviceID,IP_Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,FraudSummary,FraudRisk
0,TX000001,AC00128,14.09,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,,
1,TX000002,AC00455,376.24,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,,
2,TX000003,AC00019,126.29,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,The transaction appears to be low-risk as it i...,Low
3,TX000004,AC00070,184.50,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,The transaction appears to be legitimate as it...,Low
4,TX000005,AC00411,13.45,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,TX002508,AC00297,856.21,Credit,Colorado Springs,D000625,21.157.41.17,M072,Branch,33,Doctor,109,1,12690.79,"The transaction appears to be legitimate, as i...",Low
2508,TX002509,AC00322,251.54,Debit,Tucson,D000410,49.174.157.140,M029,Branch,48,Doctor,177,1,254.75,The transaction shows signs of potential risk ...,Medium
2509,TX002510,AC00095,28.63,Debit,San Diego,D000095,58.1.27.124,M087,Branch,56,Retired,146,1,3382.91,The transaction appears to be legitimate as it...,Low
2510,TX002511,AC00118,185.97,Debit,Denver,D000634,21.190.11.223,M041,Online,23,Student,19,1,1776.91,,


In [ ]:
df.isnull().sum()

TransactionID          0
AccountID              0
TransactionAmount      0
TransactionType        0
Location               0
DeviceID               0
IP_Address             0
MerchantID             0
Channel                0
CustomerAge            0
CustomerOccupation     0
TransactionDuration    0
LoginAttempts          0
AccountBalance         0
FraudSummary           0
FraudRisk              0
dtype: int64

In [ ]:
df["FraudSummary"]

0                                                        
1                                                        
2       The transaction appears to be low-risk as it i...
3       The transaction appears to be legitimate as it...
4                                                        
                              ...                        
2507    The transaction appears to be legitimate, as i...
2508    The transaction shows signs of potential risk ...
2509    The transaction appears to be legitimate as it...
2510                                                     
2511    The transaction exhibits potential risk due to...
Name: FraudSummary, Length: 2512, dtype: object

In [ ]:
df["FraudRisk"]

0             
1             
2          Low
3          Low
4             
         ...  
2507       Low
2508    Medium
2509       Low
2510          
2511    Medium
Name: FraudRisk, Length: 2512, dtype: object

In [ ]:
df

,TransactionID,AccountID,TransactionAmount,TransactionType,Location,DeviceID,IP_Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,FraudSummary,FraudRisk
0,TX000001,AC00128,14.09,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,The transaction appears to be legitimate as it...,Low
1,TX000002,AC00455,376.24,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,The transaction appears to be legitimate based...,Low
2,TX000003,AC00019,126.29,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,The transaction appears to be low-risk as it i...,Low
3,TX000004,AC00070,184.50,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,The transaction appears to be legitimate as it...,Low
4,TX000005,AC00411,13.45,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,The transaction appears to be low risk as it i...,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,TX002508,AC00297,856.21,Credit,Colorado Springs,D000625,21.157.41.17,M072,Branch,33,Doctor,109,1,12690.79,"The transaction appears to be legitimate, as i...",Low
2508,TX002509,AC00322,251.54,Debit,Tucson,D000410,49.174.157.140,M029,Branch,48,Doctor,177,1,254.75,The transaction shows signs of potential risk ...,Medium
2509,TX002510,AC00095,28.63,Debit,San Diego,D000095,58.1.27.124,M087,Branch,56,Retired,146,1,3382.91,The transaction appears to be legitimate as it...,Low
2510,TX002511,AC00118,185.97,Debit,Denver,D000634,21.190.11.223,M041,Online,23,Student,19,1,1776.91,,


In [ ]:
# Only Reprocess Truly Blank Results 
blank_rows = df[
    (df["FraudSummary"].isnull()) | (df["FraudSummary"].str.strip() == "") |
    (df["FraudRisk"].isnull())    | (df["FraudRisk"].str.strip() == "")
]

print(f"\n Reprocessing {len(blank_rows)} rows with missing GPT results...")

for idx, row in blank_rows.iterrows():
    transaction_dict = row.to_dict()
    try:
        result = analyze_transaction(transaction_dict)
        df.at[idx, "FraudSummary"] = result["FraudSummary"]
        df.at[idx, "FraudRisk"] = result["FraudRisk"]
        print(f"[Fixed] Row {idx} successfully reprocessed.")
    except Exception as e:
        print(f"[Still Failed] Row {idx} after retry: {e}")
    time.sleep(1.2)



🔁 Reprocessing 1 rows with missing GPT results...


[Fixed] Row 1501 successfully reprocessed.


In [ ]:
df

,TransactionID,AccountID,TransactionAmount,TransactionType,Location,DeviceID,IP_Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,FraudSummary,FraudRisk
0,TX000001,AC00128,14.09,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,The transaction appears to be legitimate as it...,Low
1,TX000002,AC00455,376.24,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,The transaction appears to be legitimate based...,Low
2,TX000003,AC00019,126.29,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,The transaction appears to be low-risk as it i...,Low
3,TX000004,AC00070,184.50,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,The transaction appears to be legitimate as it...,Low
4,TX000005,AC00411,13.45,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,The transaction appears to be low risk as it i...,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,TX002508,AC00297,856.21,Credit,Colorado Springs,D000625,21.157.41.17,M072,Branch,33,Doctor,109,1,12690.79,"The transaction appears to be legitimate, as i...",Low
2508,TX002509,AC00322,251.54,Debit,Tucson,D000410,49.174.157.140,M029,Branch,48,Doctor,177,1,254.75,The transaction shows signs of potential risk ...,Medium
2509,TX002510,AC00095,28.63,Debit,San Diego,D000095,58.1.27.124,M087,Branch,56,Retired,146,1,3382.91,The transaction appears to be legitimate as it...,Low
2510,TX002511,AC00118,185.97,Debit,Denver,D000634,21.190.11.223,M041,Online,23,Student,19,1,1776.91,The transaction appears to be typical for a st...,Low


In [ ]:
df.to_csv("fraud_results_final.csv", index=False)

# Step 3: Mask extremely sensitive columns
Only mask high-risk fields like IDs and raw PII:

In [ ]:
df = df.copy()

# Masking only extremely sensitive columns
df["TransactionID"] = "TXN-" + df.index.astype(str).str.zfill(6)
df["AccountID"] = "ACCT-XXXX"
df["IP_Address"] = "XXX.XXX.XXX.XXX"
df["DeviceID"] = df["DeviceID"].apply(lambda x: f"device_{hash(x) % 10000}")

In [ ]:
df.head()

,TransactionID,AccountID,TransactionAmount,TransactionType,Location,DeviceID,IP_Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,FraudSummary,FraudRisk
0,TXN-000000,ACCT-XXXX,14.09,Debit,San Diego,device_3694,XXX.XXX.XXX.XXX,M015,ATM,70,Doctor,81,1,5112.21,The transaction appears to be legitimate as it...,Low
1,TXN-000001,ACCT-XXXX,376.24,Debit,Houston,device_8643,XXX.XXX.XXX.XXX,M052,ATM,68,Doctor,141,1,13758.91,The transaction appears to be legitimate based...,Low
2,TXN-000002,ACCT-XXXX,126.29,Debit,Mesa,device_8769,XXX.XXX.XXX.XXX,M009,Online,19,Student,56,1,1122.35,The transaction appears to be low-risk as it i...,Low
3,TXN-000003,ACCT-XXXX,184.50,Debit,Raleigh,device_5920,XXX.XXX.XXX.XXX,M002,Online,26,Student,25,1,8569.06,The transaction appears to be legitimate as it...,Low
4,TXN-000004,ACCT-XXXX,13.45,Credit,Atlanta,device_2465,XXX.XXX.XXX.XXX,M091,Online,26,Student,198,1,7429.40,The transaction appears to be low risk as it i...,Low


### Step 4: Save to Lakehouse → Files → Gold

In [ ]:
import os

output_path = "/lakehouse/default/Files/Gold/fraud_results_masked.csv"
output_dir = os.path.dirname(output_path)

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
df.to_csv(output_path, index=False)

Ready for Power BI import or sharing.